<a href="https://colab.research.google.com/github/rebecca-gruver/rebecca-gruver/blob/master/ais_data_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/AIS_2020_01_01.csv'
df = pd.read_csv(path)

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,367149340,2020-01-01T00:00:00,29.96476,-90.02724,1.3,10.0,16.0,SYDNEE TAYLOR,NaN,WDD4807,31.0,0.0,26.0,9.0,NaN,31.0,B
1,367687520,2020-01-01T00:00:00,30.20558,-91.03578,10.7,124.9,130.0,CHIPPEWA,NaN,WDI3361,31.0,0.0,22.0,NaN,NaN,NaN,B
2,367368170,2020-01-01T00:00:03,47.53785,-122.32833,0.6,46.5,141.0,SONJA H,NaN,WDE5536,31.0,0.0,18.0,6.0,NaN,32.0,B
3,367007980,2020-01-01T00:00:05,37.95154,-121.32682,0.0,-49.6,511.0,ANGIE M BRUSCO,IMO5111359,WDC3446,31.0,0.0,28.0,7.0,3.4,NaN,B
4,367538940,2020-01-01T00:00:05,30.00258,-93.22608,3.1,168.0,511.0,RITA ANN,NaN,WDG4670,31.0,0.0,21.0,NaN,NaN,NaN,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7033450,366967640,2020-01-01T23:59:50,38.54004,-90.24983,0.0,164.1,511.0,CINDY L,NaN,WDB8999,31.0,15.0,12.0,6.0,NaN,31.0,B
7033451,367092150,2020-01-01T23:59:52,56.81356,-132.96105,0.0,154.0,511.0,ISLAND GIRL,NaN,WDC8725,30.0,NaN,14.0,4.0,NaN,NaN,B
7033452,366986330,2020-01-01T23:59:52,38.68214,-90.19468,0.4,-49.6,511.0,MISS SHEILA,NaN,WDC2002,31.0,15.0,19.0,7.0,NaN,NaN,B
7033453,366998040,2020-01-01T23:59:55,28.41163,-96.47558,3.7,62.2,511.0,ST JOSEPH,IMO8983131,WDC2814,31.0,0.0,22.0,10.0,NaN,52.0,B


In [ ]:
df = df.dropna()

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
9,477628100,2020-01-01T00:00:01,36.80096,-75.22302,12.8,-128.7,283.0,NINGBO SEAL,IMO9579066,VRJD3,70.0,0.0,225.0,32.0,14.3,70.0,B
13,367091120,2020-01-01T00:00:04,39.89587,-75.19673,0.0,140.6,511.0,ROBERT E MCALLISTER,IMO9068574,WDC8669,31.0,0.0,33.0,9.0,4.0,52.0,B
17,538002845,2020-01-01T00:00:10,26.08420,-79.48273,13.2,189.0,187.0,YASA GOLDEN DARDANEL,IMO9339985,V7ME9,80.0,0.0,245.0,42.0,15.0,89.0,B
24,367528690,2020-01-01T00:00:02,53.90575,-166.51147,0.1,-175.8,15.0,ALASKAN LADY,IMO7742358,WDG3692,30.0,0.0,51.0,9.0,3.0,2.0,B
29,477214900,2020-01-01T00:00:06,48.13155,-123.43617,0.1,157.5,68.0,PTI HERCULES,IMO9358307,VRBY7,80.0,0.0,183.0,32.0,13.2,80.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7033378,403579000,2020-01-01T23:58:52,29.88295,-89.95680,0.0,61.0,100.0,NCC NAJD,IMO9299874,HZAK,80.0,1.0,183.0,32.0,12.2,89.0,A
7033415,338217000,2020-01-01T23:57:28,29.74080,-95.11424,0.0,43.3,45.0,LIBERTY GLORY,IMO9228136,WADP,70.0,5.0,190.0,32.0,11.9,70.0,A
7033430,368555000,2020-01-01T23:59:59,47.63040,-122.38389,0.0,-87.8,511.0,ARCTIC STORM,IMO8720113,WTQ5896,30.0,0.0,101.0,14.0,6.4,30.0,B
7033441,367602520,2020-01-01T23:58:49,32.85486,-79.95249,0.0,-190.8,52.0,MOIRA MCALLISTER,IMO8981391,WDH2920,31.0,15.0,28.0,9.0,3.4,0.0,B


In [ ]:
grouped_df = df.groupby('VesselName')

In [ ]:
grouped_lists = grouped_df['LAT'].apply(list)

In [ ]:
grouped_lists = grouped_lists.reset_index()

In [ ]:
grouped_lists

,VesselName,LAT
0,A J MCALLISTER,"[37.81453, 37.81788, 37.82447, 37.828379999999..."
1,ABIGAIL CLAIRE,"[30.49199, 30.493859999999998, 30.49751, 30.49..."
2,ACADIAN,"[42.992670000000004, 42.992740000000005, 42.99..."
3,ACHIEVEMENT,"[28.234, 28.2526, 28.163590000000003, 28.11454..."
4,ADAM F KRAUSE,"[39.269729999999996, 39.269729999999996, 39.26..."
...,...,...
1296,ZACHERY REINAUER,"[42.3804, 42.380359999999996, 42.3803599999999..."
1297,ZENITH,"[26.13944, 26.13944, 26.13944, 26.13945, 26.13..."
1298,ZIM ROTTERDAM,"[25.80712, 25.847479999999997, 25.87525, 25.88..."
1299,ZOOM ZOOM ZOOM,"[25.49713, 25.497139999999998]"


In [ ]:
all_data = df['LAT'].values.astype(float)

In [ ]:
print(all_data)

[36.80096 39.89587 26.0842  ... 47.6304  32.85486 34.28028]


In [ ]:
len(all_data)

788157

In [ ]:
all_data

array([36.80096, 39.89587, 26.0842 , ..., 47.6304 , 32.85486, 34.28028])

In [ ]:
test_data_size = 8157

In [ ]:
train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

In [ ]:
print(len(train_data))

780000


In [ ]:
print(train_data)

[36.80096 39.89587 26.0842  ... 41.97533 40.53365 29.81467]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1,1))

In [ ]:
print(train_data_normalized[:5])
print(train_data_normalized[-5:])

[[-0.06130338]
 [ 0.07383373]
 [-0.5292433 ]
 [ 0.68556545]
 [ 0.433439  ]]
[[-0.19480351]
 [-0.3617636 ]
 [ 0.16463191]
 [ 0.10168195]
 [-0.3663549 ]]


In [ ]:
import torch
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1).cuda()

In [ ]:
train_window = 24

In [ ]:
def create_inout_sequences(input_data, tw):
  inout_seq = []
  L = len(input_data)
  for i in range(L-tw):
    train_seq = input_data[i:i+tw]
    train_label = input_data[i+tw:i+tw+1]
    inout_seq.append((train_seq, train_label))
  return inout_seq 

In [ ]:
train_inout_seq = create_inout_sequences(train_data_normalized,train_window)

In [ ]:
train_inout_seq[:5]

[(tensor([-0.0613,  0.0738, -0.5292,  0.6856,  0.4334, -0.0194, -0.0597, -0.3869,
          -0.0190,  0.0433,  0.3251, -0.4939,  0.4689,  0.0743, -0.3620,  0.4126,
           0.1016,  0.1063, -0.5261, -0.3697,  0.4364, -0.2336,  0.0542, -0.4111],
         device='cuda:0'), tensor([-0.3487], device='cuda:0')),
 (tensor([ 0.0738, -0.5292,  0.6856,  0.4334, -0.0194, -0.0597, -0.3869, -0.0190,
           0.0433,  0.3251, -0.4939,  0.4689,  0.0743, -0.3620,  0.4126,  0.1016,
           0.1063, -0.5261, -0.3697,  0.4364, -0.2336,  0.0542, -0.4111, -0.3487],
         device='cuda:0'),
  tensor([0.1498], device='cuda:0')),
 (tensor([-0.5292,  0.6856,  0.4334, -0.0194, -0.0597, -0.3869, -0.0190,  0.0433,
           0.3251, -0.4939,  0.4689,  0.0743, -0.3620,  0.4126,  0.1016,  0.1063,
          -0.5261, -0.3697,  0.4364, -0.2336,  0.0542, -0.4111, -0.3487,  0.1498],
         device='cuda:0'), tensor([-0.4886], device='cuda:0')),
 (tensor([ 0.6856,  0.4334, -0.0194, -0.0597, -0.3869, -0.0190,  0

In [ ]:
import torch.nn as nn
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
model = LSTM()
model = model.cuda()
loss_function = nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:

epochs = 150
for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).cuda(),
                        torch.zeros(1, 1, model.hidden_layer_size).cuda())

        y_pred = model(seq.cuda())

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),'model_state_'+str(epochs)+'.pt')

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.05476350


In [ ]:
fut_pred = 24

test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)

[-0.3622683584690094, -0.3537433445453644, -0.3694070279598236, -0.5107496976852417, -0.3562968373298645, -0.05232905596494675, -0.3560138940811157, -0.5002711415290833, -0.36337044835090637, 0.10816565901041031, -0.36153873801231384, 0.1434551626443863, -0.5132359862327576, -0.5131045579910278, -0.3406750559806824, -0.34886953234672546, -0.34886953234672546, -0.3908379077911377, -0.3693402409553528, -0.19480350613594055, -0.361763596534729, 0.1646319031715393, 0.10168194770812988, -0.36635491251945496]


In [ ]:
model.eval()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

In [ ]:
test_inputs[fut_pred:]

In [ ]:
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,367149340,2020-01-01T00:00:00,29.96476,-90.02724,1.3,10.0,16.0,SYDNEE TAYLOR,NaN,WDD4807,31.0,0.0,26.0,9.0,NaN,31.0,B
1,367687520,2020-01-01T00:00:00,30.20558,-91.03578,10.7,124.9,130.0,CHIPPEWA,NaN,WDI3361,31.0,0.0,22.0,NaN,NaN,NaN,B
2,367368170,2020-01-01T00:00:03,47.53785,-122.32833,0.6,46.5,141.0,SONJA H,NaN,WDE5536,31.0,0.0,18.0,6.0,NaN,32.0,B
3,367007980,2020-01-01T00:00:05,37.95154,-121.32682,0.0,-49.6,511.0,ANGIE M BRUSCO,IMO5111359,WDC3446,31.0,0.0,28.0,7.0,3.4,NaN,B
4,367538940,2020-01-01T00:00:05,30.00258,-93.22608,3.1,168.0,511.0,RITA ANN,NaN,WDG4670,31.0,0.0,21.0,NaN,NaN,NaN,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229017,367106880,2020-01-01T00:50:18,30.28373,-91.22243,5.2,-152.8,276.0,NEW ROADS,IMO7527980,WDC9803,60.0,0.0,45.0,20.0,1.5,NaN,A
229018,369339000,2020-01-01T00:50:12,33.73357,-118.25167,0.0,-158.6,251.0,PRESIDENT KENNEDY,IMO9295218,WPKW,79.0,0.0,300.0,42.0,11.2,79.0,B
229019,244203000,2020-01-01T00:51:47,27.84289,-97.07279,0.0,-56.9,258.0,HAPPY BUCCANEER,IMO8300389,PEND,70.0,5.0,145.0,28.0,8.3,NaN,A
229020,338162563,2020-01-01T00:51:50,30.36263,-89.08688,0.0,3.9,511.0,BLUE MOON,NaN,NaN,37.0,NaN,14.0,NaN,NaN,NaN,B


In [ ]:
import torch
import torch.nn as nn 
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,367149340,2020-01-01T00:00:00,29.96476,-90.02724,1.3,10.0,16.0,SYDNEE TAYLOR,NaN,WDD4807,31.0,0.0,26.0,9.0,NaN,31.0,B
1,367687520,2020-01-01T00:00:00,30.20558,-91.03578,10.7,124.9,130.0,CHIPPEWA,NaN,WDI3361,31.0,0.0,22.0,NaN,NaN,NaN,B
2,367368170,2020-01-01T00:00:03,47.53785,-122.32833,0.6,46.5,141.0,SONJA H,NaN,WDE5536,31.0,0.0,18.0,6.0,NaN,32.0,B
3,367007980,2020-01-01T00:00:05,37.95154,-121.32682,0.0,-49.6,511.0,ANGIE M BRUSCO,IMO5111359,WDC3446,31.0,0.0,28.0,7.0,3.4,NaN,B
4,367538940,2020-01-01T00:00:05,30.00258,-93.22608,3.1,168.0,511.0,RITA ANN,NaN,WDG4670,31.0,0.0,21.0,NaN,NaN,NaN,B


In [ ]:
df.shape

(229022, 17)

In [ ]:
df.columns

Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status', 'Length',
       'Width', 'Draft', 'Cargo', 'TranscieverClass'],
      dtype='object')

In [ ]:
229022-200000

29022

In [ ]:
vessel_data = df['VesselName']

In [ ]:
test_data_size = 29022
train_data = vessel_data[:-test_data_size]
test_data = vessel_data[-test_data_size:]

In [ ]:
print(len(train_data))
print(len(test_data))

200000
29022


In [ ]:
print(test_data)

200000            DAUNTLESS
200001       MISS GABRIELLA
200002            UNBRIDLED
200003              JAMES T
200004         CG BARRACUDA
                ...        
229017            NEW ROADS
229018    PRESIDENT KENNEDY
229019      HAPPY BUCCANEER
229020            BLUE MOON
229021                  NaN
Name: VesselName, Length: 29022, dtype: object
